# Referencing Task & Output

### Setup

In [1]:
# !pip install crewai
# !pip install crewai-tools

In [2]:
# ollama pull llama3.2:1b

### Imports

In [3]:
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
from pydantic import BaseModel

from crewai import Agent, Task, Crew
import json

### Setting Pydantic Models

In [5]:
class ResearchFindings(BaseModel):
    """Structured research report output"""
    title: str
    key_findings: list[str]

class AnalysisSummary(BaseModel):
    """Structured summary of research findings"""
    insights: list[str]
    key_takeaways: str

### Define Agents

In [6]:
from crewai import Agent

research_agent = Agent(
    role = "AI Researcher",
    goal = "Find and summarize the latest AI advancements",
    backstory = """You are an expert AI researcher who
                stays up to date with the latest innovations.""",
    verbose = True,
)

analysis_agent = Agent(
    role = "AI Analyst",
    goal = "Analyze AI research findings and extract key insights",
    backstory = """You are a data analyst who extracts
                valuable insights from research data.""",
    verbose = True,
)

writer_agent = Agent(
    role = "Tech Writer",
    goal = "Write a well-structured blog post on AI trends",
    backstory = """You are a technology writer skilled at
                transforming complex AI research into
                readable content.""",
    verbose = True,
)

### Define Guardrails

In [7]:
def capp_insight_length(task_output):
    try:
        outputs = json.loads(task_output.pydantic.model_dump_json())["insights"]
        output = " ".join(outputs)
        if len(output) == 0:
            return (False, "Insight is empty")
        
        if len(output.split()) > 100:
            return (False, "Insight must be less than 100 words")
        
        return (True, output)
    except:
        return(False, "Invalid Json")

### Define Task

In [8]:
from crewai import Task

research_task = Task(
    description = "Find and summarize the latest AI advancements",
    expected_output = "A structured list of recent AI breakthroughs",
    agent = research_agent,
    output_pydantic = ResearchFindings
)

analysis_task = Task(
    description = "Analyze AI research findings and extract key insights in less than 100 words",
    expected_output = "A structured summary with key takeaways",
    agent = analysis_agent,
    output_pydantic = AnalysisSummary,
    guardrail = capp_insight_length
)

# Step 3: Blog Writing Task (References Both Research and Analysis)
blog_writing_task = Task(
    description = "Write a detailed blog post about AI trends",
    expected_output = "A well-structured blog post",
    agent = writer_agent,
    context = [research_task, analysis_task] 
)

**NOTE**

While we could have explicitly specified that the analysis task depends on research findings using this → context=[research_task] (shown below), it is not needed since by default, a task always references the output of the previous task:



In [9]:
ai_research_crew = Crew(
    agents=[research_agent, analysis_agent, writer_agent],
    tasks=[research_task, analysis_task, blog_writing_task],
    verbose=True
)

result = ai_research_crew.kickoff()

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 38bb6866-51fe-4372-8a66-76482feb30fb                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: f4c3a244-b1b7-4c80-b8a5-997f15800aff
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: f4c3a244-b1b7-4c80-b8a5-997f15800aff
       Status: Executing Task...
    └── 🤖 Agent: AI Researcher
            Status: In Progress

# Agent: AI Researcher
## Task: Find and summarize the latest AI advancements


🤖 Agent: AI Researcher
    Status: In Progress
└── 🧠 Thinking...

...

# Agent: Tech Writer
## Task: Write a detailed blog post about AI trends


# Agent: Tech Writer
## Final Answer: 
# Recent AI Breakthroughs in 2023

Artificial Intelligence (AI) continues to transform the world around us, and as we delve into 2023, the landscape of AI reflects remarkable advancements. This blog post will explore some key trends and breakthroughs in AI that are shaping various industries while enhancing human experiences. Here’s a detailed look at the most impactful developments thus far.

## 1. Enhanced Conversational Abilities with ChatGPT-4

One of the standout advances in 2023 is the introduction of ChatGPT-4, a significant leap forward in natural language processing. This model showcases enhanced contextual understanding, which allows it to offer more accurate responses and improved conversational abilities. Unlike its predecessors, ChatGPT-4 can maintain a coherent dialogue over longer interactions, making it a valuable tool for customer service, virtual assistanc

🚀 Crew: crew
├── 📋 Task: f4c3a244-b1b7-4c80-b8a5-997f15800aff
│      Assigned to: AI Researcher
│      Status: ✅ Completed
│   └── 🤖 Agent: AI Researcher
│           Status: ✅ Completed
├── 📋 Task: 5ba0c39a-643f-4143-aad6-22467123ab2e
│      Assigned to: AI Analyst
│      Status: ✅ Completed
│   └── 🤖 Agent: AI Analyst
│           Status: ✅ Completed
└── 📋 Task: 16c298e7-274e-40d6-a61f-311a6bf9e3ed
       Status: Executing Task...
    └── 🤖 Agent: Tech Writer
            Status: ✅ Completed

🚀 Crew: crew
├── 📋 Task: f4c3a244-b1b7-4c80-b8a5-997f15800aff
│      Assigned to: AI Researcher
│      Status: ✅ Completed
│   └── 🤖 Agent: AI Researcher
│           Status: ✅ Completed
├── 📋 Task: 5ba0c39a-643f-4143-aad6-22467123ab2e
│      Assigned to: AI Analyst
│      Status: ✅ Completed
│   └── 🤖 Agent: AI Analyst
│           Status: ✅ Completed
└── 📋 Task: 16c298e7-274e-40d6-a61f-311a6bf9e3ed
       Assigned to: Tech Writer
       Status: ✅ Completed
    └── 🤖 Agent: Tech Writer
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 16c298e7-274e-40d6-a61f-311a6bf9e3ed                                                                     │
│  Agent: Tech Writer                                                                                             │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 38bb6866-51fe-4372-8a66-76482feb30fb                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [11]:
print("\n --- Generated Blog Post ---")
from IPython.display import Markdown
Markdown(result.raw)


 --- Generated Blog Post ---


# Recent AI Breakthroughs in 2023

Artificial Intelligence (AI) continues to transform the world around us, and as we delve into 2023, the landscape of AI reflects remarkable advancements. This blog post will explore some key trends and breakthroughs in AI that are shaping various industries while enhancing human experiences. Here’s a detailed look at the most impactful developments thus far.

## 1. Enhanced Conversational Abilities with ChatGPT-4

One of the standout advances in 2023 is the introduction of ChatGPT-4, a significant leap forward in natural language processing. This model showcases enhanced contextual understanding, which allows it to offer more accurate responses and improved conversational abilities. Unlike its predecessors, ChatGPT-4 can maintain a coherent dialogue over longer interactions, making it a valuable tool for customer service, virtual assistance, and educational purposes.

## 2. Creativity Boosted by Generative Models like DALL-E 2

The evolution of generative models continues with DALL-E 2, which empowers users to create and manipulate images from natural language descriptions. This capability not only makes artistic creation more accessible but also enhances creative workflows in industries like marketing, design, and entertainment. The availability of such innovative tools drives collaboration between human creativity and machine efficiency.

## 3. Accelerating Drug Discovery with AI

In the healthcare sector, AI has revolutionized the process of drug discovery. AI-powered platforms are significantly reducing the time needed to identify new compounds, streamlining clinical trials and bringing potential cures to market faster than ever before. These platforms leverage advanced algorithms to analyze vast datasets, identifying promising candidates that would traditionally take years to uncover.

## 4. Breakthroughs in Robotics through Reinforcement Learning

Reinforcement learning continues to advance, resulting in significant breakthroughs in robotics. Machines are now better equipped to learn complex tasks within dynamic environments, paving the way for autonomous systems that can adapt to unexpected changes. This development holds promise for applications in manufacturing, logistics, and even daily life as robots become more capable of assisting humans.

## 5. Federated Learning: Collaborative and Private

Privacy concerns are increasingly vital in AI development, leading to the introduction of federated learning. This technique allows multiple devices to collaborate on machine learning without sharing sensitive data. By maintaining privacy while improving model accuracy, federated learning can empower industries such as healthcare and finance to harness AI without compromising user trust or compliance.

## 6. Enhancing Transparency with Explainable AI (XAI)

As AI systems become more integral to decision-making, the demand for transparency has surged. Recent advancements in Explainable AI (XAI) enhance the interpretability of AI models, allowing users to understand how decisions are made. This is crucial not only for regulatory compliance but also for building trust in AI applications across sectors such as finance, healthcare, and law enforcement.

## 7. Multi-Modal AI Systems for Rich User Experiences

The development of robust multi-modal AI systems marks a pivotal trend in improving user experiences. These systems seamlessly integrate information from various inputs, including text, images, and audio, creating more interactive and enriched interactions. For example, virtual personal assistants can now respond to voice commands while recognizing visual cues, making them more effective and user-friendly.

## 8. AI Innovations in Climate Modeling

AI is playing a crucial role in environmental protection and sustainability efforts. Advanced analytics are being utilized in climate modeling and prediction, aiding scientists and policymakers in understanding complex environmental challenges. These AI-driven insights help address threats such as climate change while fostering sustainable practices across various industries.

## 9. Personalized Education Tools

In the realm of education, AI-based personalized learning tools are reshaping the way students engage with content. These tools analyze individual learning patterns to provide customized learning experiences, adapting to each student's needs and performance. This targeted approach not only enhances educational outcomes but also promotes inclusivity within the classroom.

## 10. Advances in AI Ethics

Finally, the growing focus on AI ethics has prompted significant improvements in frameworks and guidelines. Efforts to reduce bias and ensure fair representation in AI systems are gaining traction, fostering a more equitable technological landscape. As AI continues to permeate various facets of life, these ethical considerations are vital for driving responsible innovation.

### Conclusion

The trends and breakthroughs in AI during 2023 signify a promising future where technology enhances creative expression, aids in healthcare advancements, and supports sustainable practices while prioritizing ethical considerations. As these innovations mature, they will redefine how we interact with technology, opening doors to countless possibilities for improving lives across the globe. Keeping an eye on these developments will be essential for anyone interested in the future of AI.

In summary, as we progress further into 2023, the AI landscape is not just about technological prowess but also about creating meaningful impacts on society. Stay tuned for more updates as we continue to explore these exciting advancements!

### Explicitly Access Output of Task

In [21]:
analysis_task.output

TaskOutput(description='Analyze AI research findings and extract key insights in less than 100 words', name=None, expected_output='A structured summary with key takeaways', summary='Analyze AI research findings and extract key insights in less...', raw='Enhanced contextual understanding in ChatGPT-4 improves conversational abilities. Generative models like DALL-E 2 enable creative image generation from natural language. AI-powered drug discovery accelerates new compound identification and clinical trials. Reinforcement learning breakthroughs enhance robotic task learning in dynamic environments. Federated learning facilitates collaborative ML while preserving data privacy. Explainable AI (XAI) improves transparency and compliance in AI systems. Multi-modal AI integrates text, images, and audio for richer experiences. AI innovations support climate modeling and sustainable practices. Personalized education tools enhance tailored adaptive learning. Ethics frameworks improve bias reductio